In [11]:
import os
import pandas as pd

In [12]:
df_id = pd.read_csv(os.path.join(os.getcwd(), r"AIBL-Total.csv"))

df_id_not_type = df_id[~df_id["Imaging Protocol"].str.contains("Type")]
df_id_not_type["Imaging Protocol"] = (
    df_id_not_type["Imaging Protocol"] + ";Acquisition Type=3D"
)
df_id_type = df_id[df_id["Imaging Protocol"].str.contains("Type")]
df_id = pd.concat([df_id_not_type, df_id_type])


df_id[["Manufacturer", "Model", "Type"]] = df_id["Imaging Protocol"].str.extract(
    "Manufacturer=(.+);Mfg Model=(.+);Acquisition Type=(.+)", expand=True
)
df_id["Type"] = df_id["Type"].fillna("3D")
df_id = df_id[df_id["Type"] == "3D"]

df_id = df_id.replace(
    {
        "GE MEDICAL SYSTEMS": "GE",
        "Philips Healthcare": "Philips",
        "Philips Medical Systems": "Philips",
        "SIEMENS|PixelMed": "Siemens",
        "SIEMENS": "Siemens",
    },
    regex=False,
)

df_id = df_id[df_id["Description"].str.contains("MP")]

In [13]:
df_id.head()

,Subject ID,Sex,Research Group,Visit,Age,Description,Imaging Protocol,Image ID,Manufacturer,Model,Type
3,151083,X,Phantom,Baseline,34.0,MPRAGE MODIFIED,Acquisition Plane=SAGITTAL;Manufacturer=SIEMEN...,139800,Siemens,TrioTim;Weighting=T1,3D
4,151083,X,Phantom,Baseline,34.0,MPRAGE MODIFIED CORONAL,Acquisition Plane=CORONAL;Manufacturer=SIEMENS...,139801,Siemens,TrioTim;Weighting=T1,3D
5,151083,X,Phantom,Baseline,34.0,MPRAGE MODIFIED REPEAT,Acquisition Plane=SAGITTAL;Manufacturer=SIEMEN...,139802,Siemens,TrioTim;Weighting=T1,3D
6,10,M,Patient,18 Month follow-up,83.0,MPRAGE ADNI confirmed,Acquisition Plane=SAGITTAL;Manufacturer=SIEMEN...,164086,Siemens,TrioTim;Weighting=T1,3D
11,100,M,Patient,Baseline,77.0,MPRAGE ADNI confirmed,Acquisition Plane=SAGITTAL;Manufacturer=SIEMEN...,133615,Siemens,TrioTim;Weighting=T1,3D


In [4]:
df_id["Image ID"].count()

1299

In [5]:
firsts = df_id.groupby(["Subject ID", "Visit"], as_index=False).first()

In [35]:
firsts["Subject ID"].nunique()

692

In [6]:
firsts.Visit.unique()

array(['Baseline', '18 Month follow-up', '36 Month follow-up',
       '54 Month follow-up', '72 Month follow-up'], dtype=object)

In [7]:
firsts["Image ID"].nunique()

1275

In [8]:
firsts.groupby(["Manufacturer", "Research Group"])["Image ID"].nunique()

Manufacturer  Research Group
Siemens       Patient           1274
              Phantom              1
Name: Image ID, dtype: int64

In [9]:
firsts["dot"] = 1

In [10]:
firsts[["Image ID", "dot"]].to_csv("ID.csv", index=False)